In [ ]:
from models.ensemble_model import Ensemble
from utils.data_reader import read_data

# initialize ensemble model
model = Ensemble()

# read in the data
data = read_data()
# training data is everything but the iliad
# NOTE: should almost certainly change this, this is just to get us started
training_data = { auth: texts[1:] for auth, texts in data.items() }

# NOTE: currently crashes since models are incomplete
model.train(training_data)
